In [153]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tp2datostrain/test.csv
/kaggle/input/tp2datostrain/train.csv


In [154]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from category_encoders import CountEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from category_encoders import CatBoostEncoder

In [155]:
posta = pd.read_csv("/kaggle/input/tp2datostrain/train.csv")
test = pd.read_csv("/kaggle/input/tp2datostrain/test.csv")

In [156]:
es_para_submit = False
if es_para_submit: 
    TAMANIO_TRAIN = 240000
    TAMANIO_TEST = 60000
else:
    TAMANIO_TRAIN = 190000
    TAMANIO_TEST = 50000

In [157]:
if not es_para_submit:
    test = posta.tail(TAMANIO_TEST).copy()
    train = posta.head(TAMANIO_TRAIN).copy()
    test.drop(columns = ['precio'], inplace = True)

In [158]:
dfCheto = pd.concat([train,test], sort=False)
dfCheto['precio'] = SimpleImputer(strategy='median').fit_transform(dfCheto[['precio']])
df = dfCheto.copy()

In [159]:
df.provincia.fillna('provinciaVacia', inplace = True)
aux2 = df[['ciudad', 'provincia']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby(['provincia']).agg(lambda x:x.provincia.value_counts().index[0]).reset_index().set_index('ciudad')
ciuds = aux2.T.to_dict('records').copy()
ciuds = ciuds[0]
df.ciudad.fillna(df.provincia.map(ciuds), inplace = True)
df.ciudad.fillna('ciudadVacia', inplace=True)

In [160]:
df.tipodepropiedad.fillna('tipoVacio', inplace=True)

In [161]:
aux2 = df[['idzona', 'ciudad']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby('ciudad').agg({'idzona':'median'})
ids = aux2.T.to_dict('records').copy()
ids = ids[0]
df.idzona.fillna(df.ciudad.map(ids), inplace = True)
df['idzona'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['idzona']])

In [162]:
aux = df[['lat', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lat':'mean'})
lats = aux.T.to_dict('records').copy()
lats = lats[0]
df.lat.fillna(df.ciudad.map(lats), inplace = True)
df['lat'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lat']])

In [163]:
aux = df[['lng', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lng':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.lng.fillna(df.ciudad.map(lngs), inplace = True)
df['lng'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lng']])

In [164]:
aux = df[['tipodepropiedad', 'habitaciones']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'habitaciones':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.habitaciones.fillna(df.tipodepropiedad.map(habts), inplace = True)
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

In [165]:
aux = df[['tipodepropiedad', 'garages']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'garages':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.garages.fillna(df.tipodepropiedad.map(habts), inplace = True)
df.garages.fillna(0, inplace=True)

In [166]:
df.drop(columns=['direccion'], inplace=True)

In [167]:
aux = df[['antiguedad', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'antiguedad':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.antiguedad.fillna(df.ciudad.map(lngs), inplace = True)
df['antiguedad'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['antiguedad']])

In [168]:
aux = df[['banos', 'tipodepropiedad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'banos':'median'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.banos.fillna(df.tipodepropiedad.map(lngs), inplace = True)
df.banos.fillna(1, inplace=True)

In [169]:
df.titulo.fillna('', inplace=True)
df.descripcion.fillna('', inplace=True)
df['texto'] = df.titulo + ' ' + df.descripcion
df['texto'] = df.texto.str.len()
df.drop(columns = ['descripcion', 'titulo'], inplace = True)

In [170]:
aux = df.copy()
aux.metrostotales.fillna(0, inplace=True)
aux.metroscubiertos.fillna(0, inplace =True)
aux['metrostotalesCorregidos'] = aux[['metrostotales', 'metroscubiertos']].max(axis=1)
aux['metroscubiertosCorregidos'] = aux[['metrostotales', 'metroscubiertos']].min(axis=1)
df['metrostotales'] = aux['metrostotalesCorregidos']
df['metroscubiertos'] = aux['metroscubiertosCorregidos']

In [171]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['dia'] = df.fecha.dt.day
df['mes'] = df.fecha.dt.month
df['anio'] = df.fecha.dt.year
df.drop(columns=['fecha'], inplace=True)

In [172]:
df['publicacionesPorCiudad'] = CountEncoder().fit_transform(df['ciudad'])

In [173]:
df['publicacionesPorProvincia'] = CountEncoder().fit_transform(df['provincia'])

In [174]:
df['publicacionesPorTipo'] = CountEncoder().fit_transform(df['tipodepropiedad'])

In [175]:
aux = df[['mes', 'anio']].copy()
aux['cantidad'] =  1
aux['cantidad'] = aux.groupby(['mes', 'anio']).transform('sum')
df['publicacionesEnMes'] = aux['cantidad']

In [176]:
aux = df[['metrostotales', 'tipodepropiedad']].copy()
aux['metrosPromedioPorTipo'] = aux.groupby('tipodepropiedad').transform('mean')
df['metrosPromedioPorTipo'] = aux['metrosPromedioPorTipo']

In [177]:
aux = df[['metrostotales', 'ciudad']].copy()
aux['metrosPromedioPorCiudad'] = aux.groupby('ciudad').transform('mean')
df['metrosPromedioPorCiudad'] = aux['metrosPromedioPorCiudad']

In [178]:
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

transfomador_numero = Pipeline(steps=[('a', SimpleImputer(strategy = 'median'))])

In [179]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['id', 'antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'texto',
       'dia', 'mes', 'anio','publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad']),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [180]:
model = xgb.XGBRegressor(learning_rate= 0.2,n_estimators= 250,scale_pos_weight= 2,max_depth= 10,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.75,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [181]:
#dtrain = xgb.DMatrix(df.drop(columns=['precio','tipodepropiedad', 'provincia', 'ciudad']).copy(), label=df.precio)
#params = {
#    'max_depth':6,
#    'min_child_weight': 1,
#    'eta':.3,
#    'subsample': 1,
#    'colsample_bytree': 1,
#    'objective':'reg:linear',
#}
#resultados_cv = xgb.cv(params,
#    dtrain,
#    seed=42,
#    nfold=5,
#    num_boost_round=2000,
#    metrics={'mae'},
#    early_stopping_rounds=10)

#resultados_cv.shape[0]

In [182]:
entrenamiento = df.head(TAMANIO_TRAIN).copy()
prueba = df.tail(TAMANIO_TEST).copy()

In [183]:
prueba.drop(columns=['precio'], inplace=True)

In [184]:
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()

In [185]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [186]:
my_pipeline.fit(train_x, train_y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[14:23:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                   

In [187]:
preds = my_pipeline.predict(train_x)

In [188]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(preds, train_y)

416435.85016163654

In [189]:
precio = posta.precio.tail(TAMANIO_TEST).copy()

In [190]:
preds = my_pipeline.predict(prueba)

In [191]:
if not es_para_submit:
    print(mean_absolute_error(preds, precio))

558536.8943374219


USO LOS PRECIOS ESTIMADOS POR 1RA VEZ

In [192]:
aux = df.tail(TAMANIO_TEST).copy()
aux.precio = preds
df = pd.concat([df.head(TAMANIO_TRAIN),aux], sort=False)

In [193]:
df['catBoostCiudad'] = CatBoostEncoder().fit_transform(df[['ciudad']], df['precio'])
df['catBoostTipo'] = CatBoostEncoder().fit_transform(df[['tipodepropiedad']], df['precio'])
df['catBoostProvincia'] = CatBoostEncoder().fit_transform(df[['provincia']], df['precio'])

In [194]:
aux = df[['precio', 'ciudad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('ciudad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorCiudad'] = df.ciudad.map(precioPromedio)

In [195]:
aux = df[['precio', 'tipodepropiedad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('tipodepropiedad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorTipo'] = df.tipodepropiedad.map(precioPromedio)

In [196]:
aux = df[['precio', 'idzona', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('idzona').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorZona'] = df.idzona.map(precioPromedio)

In [197]:
aux = df[['precio', 'mes', 'anio', 'metrostotales']].copy()
aux['mes-anio'] = aux.mes + aux.anio
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('mes-anio').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['mes-anio'] = df.mes + df.anio
df['precioPromedioM2PorMes'] = df['mes-anio'].map(precioPromedio)

In [198]:
aux = df[['id','provincia','ciudad','idzona','anio','mes','precio','metrostotales','tipodepropiedad']].copy()
aux['preciom2'] = aux.precio/aux.metrostotales
aux['trimestre'] = (aux.mes//3)+1-aux.mes//12
#aux = aux.groupby(['provincia','anio','tipodepropiedad'])['preciom2'].agg(['min','max','count','mean','median','std'])
def rellenarCon(dataframe,listaDeParametros):
    dataframe['cuenta'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('count')
    dataframe['estimacion'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('mean')
    nombre_col = 'mediana_'+'_'.join(listaDeParametros)
    dataframe[nombre_col] = dataframe.apply(lambda x: x['estimacion'] if x['cuenta'] >= 10 else 0,axis=1)

rellenarCon(aux,['anio','idzona'])
rellenarCon(aux,['idzona','mes','tipodepropiedad'])
rellenarCon(aux,['ciudad','anio','tipodepropiedad'])
rellenarCon(aux,['provincia','mes','tipodepropiedad'])
rellenarCon(aux,['trimestre','idzona'])
rellenarCon(aux,['trimestre','ciudad'])
#rellenarCon(aux,['idzona','trimestre','tipodepropiedad'])
#rellenarCon(aux,['ciudad','trimestre','tipodepropiedad'])
#rellenarCon(aux,['provincia','trimestre','tipodepropiedad'])
#rellenarCon(aux,['trimestre','tipodepropiedad'])

In [199]:
def devolverEstimacion(x,columnas):
    for i in range(len(columnas)):
        if columnas[i].find('mediana') > -1 and x[i]>0: return x[i]
    return 0

aux['estimacionM2'] = aux.apply(lambda x: devolverEstimacion(x=x,columnas=aux.columns),axis=1)
df['estimacion_preciom2'] = aux['estimacionM2']
#df['estimacion_anio_idzona'] = aux['mediana_anio_idzona']
#df['estimacion_ciudad_anio_tipodepropiedad'] = aux['mediana_ciudad_anio_tipodepropiedad']
#df['estimacion_trimestre_idzona'] = aux['mediana_trimestre_idzona']
#df['estimacion_trimestre_ciudad'] = aux['mediana_trimestre_ciudad']

In [200]:
#df.drop(columns=['mes-anio'], inplace = True)

In [201]:
entrenamiento = df.head(TAMANIO_TRAIN).copy()
prueba = df.tail(TAMANIO_TEST).copy()
prueba.drop(columns=['precio'], inplace=True)
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()

In [202]:
df.columns

Index(['id', 'tipodepropiedad', 'ciudad', 'provincia', 'antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'precio', 'texto',
       'dia', 'mes', 'anio', 'publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad', 'catBoostCiudad', 'catBoostTipo',
       'catBoostProvincia', 'precioPromedioM2PorCiudad',
       'precioPromedioM2PorTipo', 'precioPromedioM2PorZona', 'mes-anio',
       'precioPromedioM2PorMes', 'estimacion_preciom2'],
      dtype='object')

In [203]:
model = xgb.XGBRegressor(learning_rate= 0.1,n_estimators= 500,scale_pos_weight= 2,max_depth= 10,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.75,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [204]:
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['id', 'antiguedad','estimacion_preciom2',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'texto',
       'dia', 'mes', 'anio','publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad','catBoostCiudad', 'catBoostTipo',
       'catBoostProvincia','precioPromedioM2PorCiudad',
       'precioPromedioM2PorTipo', 'precioPromedioM2PorZona',
       'precioPromedioM2PorMes'
                                     ]),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [205]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor2),
                              ('model', model)
                             ])

In [206]:
my_pipeline.fit(train_x, train_y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[14:25:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                   

In [207]:
preds = my_pipeline.predict(train_x)

In [208]:
mean_absolute_error(preds, train_y)

317596.7611098273

In [209]:
preds = my_pipeline.predict(prueba)

In [210]:
if not es_para_submit:
    print(mean_absolute_error(preds, precio))

525601.8503915112


USO LOS PRECIOS ESTIMADOS POR 2DA VEZ

In [211]:
aux = df.tail(TAMANIO_TEST).copy()
aux.precio = preds
df = pd.concat([df.head(TAMANIO_TRAIN),aux], sort=False)

In [212]:
df['catBoostCiudad'] = CatBoostEncoder().fit_transform(df[['ciudad']], df['precio'])
df['catBoostTipo'] = CatBoostEncoder().fit_transform(df[['tipodepropiedad']], df['precio'])
df['catBoostProvincia'] = CatBoostEncoder().fit_transform(df[['provincia']], df['precio'])

In [213]:
aux = df[['precio', 'ciudad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('ciudad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorCiudad'] = df.ciudad.map(precioPromedio)

In [214]:
aux = df[['precio', 'tipodepropiedad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('tipodepropiedad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorTipo'] = df.tipodepropiedad.map(precioPromedio)

In [215]:
aux = df[['precio', 'idzona', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('idzona').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorZona'] = df.idzona.map(precioPromedio)

In [216]:
aux = df[['precio', 'mes', 'anio', 'metrostotales']].copy()
aux['mes-anio'] = aux.mes + aux.anio
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('mes-anio').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['mes-anio'] = df.mes + df.anio
df['precioPromedioM2PorMes'] = df['mes-anio'].map(precioPromedio)

In [217]:
aux = df[['id','provincia','ciudad','idzona','anio','mes','precio','metrostotales','tipodepropiedad']].copy()
aux['preciom2'] = aux.precio/aux.metrostotales
aux['trimestre'] = (aux.mes//3)+1-aux.mes//12
#aux = aux.groupby(['provincia','anio','tipodepropiedad'])['preciom2'].agg(['min','max','count','mean','median','std'])
def rellenarCon(dataframe,listaDeParametros):
    dataframe['cuenta'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('count')
    dataframe['estimacion'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('mean')
    nombre_col = 'mediana_'+'_'.join(listaDeParametros)
    dataframe[nombre_col] = dataframe.apply(lambda x: x['estimacion'] if x['cuenta'] >= 10 else 0,axis=1)

rellenarCon(aux,['anio','idzona'])
rellenarCon(aux,['idzona','mes','tipodepropiedad'])
rellenarCon(aux,['ciudad','anio','tipodepropiedad'])
rellenarCon(aux,['provincia','mes','tipodepropiedad'])
rellenarCon(aux,['trimestre','idzona'])
rellenarCon(aux,['trimestre','ciudad'])
#rellenarCon(aux,['idzona','trimestre','tipodepropiedad'])
#rellenarCon(aux,['ciudad','trimestre','tipodepropiedad'])
#rellenarCon(aux,['provincia','trimestre','tipodepropiedad'])
#rellenarCon(aux,['trimestre','tipodepropiedad'])

In [218]:
def devolverEstimacion(x,columnas):
    for i in range(len(columnas)):
        if columnas[i].find('mediana') > -1 and x[i]>0: return x[i]
    return 0

aux['estimacionM2'] = aux.apply(lambda x: devolverEstimacion(x=x,columnas=aux.columns),axis=1)
df['estimacion_preciom2'] = aux['estimacionM2']
#df['estimacion_anio_idzona'] = aux['mediana_anio_idzona']
#df['estimacion_ciudad_anio_tipodepropiedad'] = aux['mediana_ciudad_anio_tipodepropiedad']
#df['estimacion_trimestre_idzona'] = aux['mediana_trimestre_idzona']
#df['estimacion_trimestre_ciudad'] = aux['mediana_trimestre_ciudad']

In [219]:
#df.drop(columns=['mes-anio'], inplace = True)

In [220]:
entrenamiento = df.head(TAMANIO_TRAIN).copy()
prueba = df.tail(TAMANIO_TEST).copy()
prueba.drop(columns=['precio'], inplace=True)
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()

In [221]:
df.columns

Index(['id', 'tipodepropiedad', 'ciudad', 'provincia', 'antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'precio', 'texto',
       'dia', 'mes', 'anio', 'publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad', 'catBoostCiudad', 'catBoostTipo',
       'catBoostProvincia', 'precioPromedioM2PorCiudad',
       'precioPromedioM2PorTipo', 'precioPromedioM2PorZona', 'mes-anio',
       'precioPromedioM2PorMes', 'estimacion_preciom2'],
      dtype='object')

In [222]:
model = xgb.XGBRegressor(learning_rate= 0.04,n_estimators= 850,scale_pos_weight= 2,max_depth= 10,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.75,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [223]:
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['id', 'antiguedad','estimacion_preciom2',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'texto',
       'dia', 'mes', 'anio','publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad','catBoostCiudad', 'catBoostTipo',
       'catBoostProvincia','precioPromedioM2PorCiudad',
       'precioPromedioM2PorTipo', 'precioPromedioM2PorZona',
       'precioPromedioM2PorMes'
                                     ]),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [224]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor2),
                              ('model', model)
                             ])

In [225]:
my_pipeline.fit(train_x, train_y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[14:28:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                   

In [226]:
preds = my_pipeline.predict(train_x)

In [227]:
mean_absolute_error(preds, train_y)

323343.0268555921

In [228]:
preds = my_pipeline.predict(prueba)

In [229]:
if not es_para_submit:
    print(mean_absolute_error(preds, precio))

526354.2295857031


USO LOS PRECIOS ESTIMADOS POR 3RA VEZ

In [211]:
aux = df.tail(TAMANIO_TEST).copy()
aux.precio = preds
df = pd.concat([df.head(TAMANIO_TRAIN),aux], sort=False)

In [212]:
df['catBoostCiudad'] = CatBoostEncoder().fit_transform(df[['ciudad']], df['precio'])
df['catBoostTipo'] = CatBoostEncoder().fit_transform(df[['tipodepropiedad']], df['precio'])
df['catBoostProvincia'] = CatBoostEncoder().fit_transform(df[['provincia']], df['precio'])

In [213]:
aux = df[['precio', 'ciudad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('ciudad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorCiudad'] = df.ciudad.map(precioPromedio)

In [214]:
aux = df[['precio', 'tipodepropiedad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('tipodepropiedad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorTipo'] = df.tipodepropiedad.map(precioPromedio)

In [215]:
aux = df[['precio', 'idzona', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('idzona').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorZona'] = df.idzona.map(precioPromedio)

In [216]:
aux = df[['precio', 'mes', 'anio', 'metrostotales']].copy()
aux['mes-anio'] = aux.mes + aux.anio
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('mes-anio').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['mes-anio'] = df.mes + df.anio
df['precioPromedioM2PorMes'] = df['mes-anio'].map(precioPromedio)

In [217]:
aux = df[['id','provincia','ciudad','idzona','anio','mes','precio','metrostotales','tipodepropiedad']].copy()
aux['preciom2'] = aux.precio/aux.metrostotales
aux['trimestre'] = (aux.mes//3)+1-aux.mes//12
#aux = aux.groupby(['provincia','anio','tipodepropiedad'])['preciom2'].agg(['min','max','count','mean','median','std'])
def rellenarCon(dataframe,listaDeParametros):
    dataframe['cuenta'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('count')
    dataframe['estimacion'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('mean')
    nombre_col = 'mediana_'+'_'.join(listaDeParametros)
    dataframe[nombre_col] = dataframe.apply(lambda x: x['estimacion'] if x['cuenta'] >= 10 else 0,axis=1)

rellenarCon(aux,['anio','idzona'])
rellenarCon(aux,['idzona','mes','tipodepropiedad'])
rellenarCon(aux,['ciudad','anio','tipodepropiedad'])
rellenarCon(aux,['provincia','mes','tipodepropiedad'])
rellenarCon(aux,['trimestre','idzona'])
rellenarCon(aux,['trimestre','ciudad'])
#rellenarCon(aux,['idzona','trimestre','tipodepropiedad'])
#rellenarCon(aux,['ciudad','trimestre','tipodepropiedad'])
#rellenarCon(aux,['provincia','trimestre','tipodepropiedad'])
#rellenarCon(aux,['trimestre','tipodepropiedad'])

In [218]:
def devolverEstimacion(x,columnas):
    for i in range(len(columnas)):
        if columnas[i].find('mediana') > -1 and x[i]>0: return x[i]
    return 0

aux['estimacionM2'] = aux.apply(lambda x: devolverEstimacion(x=x,columnas=aux.columns),axis=1)
df['estimacion_preciom2'] = aux['estimacionM2']
#df['estimacion_anio_idzona'] = aux['mediana_anio_idzona']
#df['estimacion_ciudad_anio_tipodepropiedad'] = aux['mediana_ciudad_anio_tipodepropiedad']
#df['estimacion_trimestre_idzona'] = aux['mediana_trimestre_idzona']
#df['estimacion_trimestre_ciudad'] = aux['mediana_trimestre_ciudad']

In [219]:
#df.drop(columns=['mes-anio'], inplace = True)

In [220]:
entrenamiento = df.head(TAMANIO_TRAIN).copy()
prueba = df.tail(TAMANIO_TEST).copy()
prueba.drop(columns=['precio'], inplace=True)
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()

In [221]:
df.columns

Index(['id', 'tipodepropiedad', 'ciudad', 'provincia', 'antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'precio', 'texto',
       'dia', 'mes', 'anio', 'publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad', 'catBoostCiudad', 'catBoostTipo',
       'catBoostProvincia', 'precioPromedioM2PorCiudad',
       'precioPromedioM2PorTipo', 'precioPromedioM2PorZona', 'mes-anio',
       'precioPromedioM2PorMes', 'estimacion_preciom2'],
      dtype='object')

In [222]:
model = xgb.XGBRegressor(learning_rate= 0.04,n_estimators= 850,scale_pos_weight= 2,max_depth= 10,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.75,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [223]:
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['id', 'antiguedad','estimacion_preciom2',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'texto',
       'dia', 'mes', 'anio','publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad','catBoostCiudad', 'catBoostTipo',
       'catBoostProvincia','precioPromedioM2PorCiudad',
       'precioPromedioM2PorTipo', 'precioPromedioM2PorZona',
       'precioPromedioM2PorMes'
                                     ]),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [224]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor2),
                              ('model', model)
                             ])

In [225]:
my_pipeline.fit(train_x, train_y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[14:28:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                   

In [226]:
preds = my_pipeline.predict(train_x)

In [227]:
mean_absolute_error(preds, train_y)

323343.0268555921

In [228]:
preds = my_pipeline.predict(prueba)

In [229]:
if not es_para_submit:
    print(mean_absolute_error(preds, precio))

526354.2295857031


In [230]:
res = pd.DataFrame(preds, index=prueba.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)

,id,target
0,750,9.607638e+05
1,9294,6.932914e+05
2,63776,3.644582e+05
3,195548,7.004654e+06
4,86914,1.104354e+06
